<a href="https://colab.research.google.com/github/Muzznah/Amazon-Reviews-ETL/blob/master/Analysis_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
# Install Java, Spark, and Findspark
!apt-get install default-jdk > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark
# Install the Java JDK.


# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [3]:
# Connect to postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-07 17:39:09--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.77MB/s    in 0.2s    

2020-08-07 17:39:09 (4.77 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [82]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazone_review").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [83]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True,inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [6]:
# Count the number of rows.
df.count()

1752932

In [96]:
# Check latest review date
df.select('review_date').sort('review_date',ascending=False).show(2)


+-----------+
|review_date|
+-----------+
| 2015-08-31|
| 2015-08-31|
+-----------+
only showing top 2 rows



In [98]:
# Check oldest review date
df.select('review_date').filter(df['review_date']!= 'null').sort('review_date',ascending=True).show(2)

+-----------+
|review_date|
+-----------+
| 1999-07-13|
| 1999-07-13|
+-----------+
only showing top 2 rows



In [7]:
# Check data types.
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
# Drop data with NAN values
df.dropna(subset=['product_id', 'customer_id', 'review_id'])
df.count()

1752932

In [9]:
# Check for duplicates.
df.drop_duplicates(['customer_id','product_id','review_id'])
df.count()

1752932

# Analysis

In [99]:
# Filter for verified purchases.
cleaned_df=df.filter(df['verified_purchase']== "Y")
cleaned_df.count()

1392128

In [100]:
# Check oldest review date
cleaned_df.select('review_date').filter(cleaned_df['review_date']!= 'null').sort('review_date',ascending=True).show(2)

+-----------+
|review_date|
+-----------+
| 1999-08-13|
| 1999-10-16|
+-----------+
only showing top 2 rows



In [101]:
# Check Latest review date
cleaned_df.select('review_date').filter(cleaned_df['review_date']!= 'null').sort('review_date',ascending=False).show(2)

+-----------+
|review_date|
+-----------+
| 2015-08-31|
| 2015-08-31|
+-----------+
only showing top 2 rows



In [11]:
# Calculate V vs !V review count for df(not just verified purchase).
df.groupBy('vine').count().show()

+----+-------+
|vine|  count|
+----+-------+
|   Y|  12100|
|   N|1740832|
+----+-------+



In [12]:
# Calculate V vs !V review count for verified purchase df.
cleaned_df.groupBy('vine').count().show()

+----+-------+
|vine|  count|
+----+-------+
|   Y|     19|
|   N|1392109|
+----+-------+



In [18]:
# Calculate avergae star rating for V vs !V.
avg_rating_df= df.groupBy('vine').mean('star_rating')
avg_rating_df.show()

+----+-----------------+
|vine| avg(star_rating)|
+----+-----------------+
|   Y|4.290082644628099|
|   N| 4.16179734747523|
+----+-----------------+



In [19]:
avg_rating_verified= cleaned_df.groupBy('vine').mean('star_rating')
avg_rating_verified.show()

+----+-----------------+
|vine| avg(star_rating)|
+----+-----------------+
|   Y|4.157894736842105|
|   N|4.236202050270489|
+----+-----------------+



In [21]:
# Check how helpful the V vs !V reviews were.
cleaned_df.groupBy('vine').mean('helpful_votes').show()

+----+------------------+
|vine|avg(helpful_votes)|
+----+------------------+
|   Y| 3.473684210526316|
|   N|1.1019237717736183|
+----+------------------+



In [29]:
#Import an sql function to use col.
from pyspark.sql.functions import col
# from pyspark.sql.types import DateType

cleaned_df=cleaned_df.withColumn("review_date",col("review_date").cast('date'))

In [42]:
# Create df for review_id_table.
review_id_df=cleaned_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show(5)
review_id_df.count()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



1392128

In [43]:
# Drop duplicates.
review_table=review_id_df.dropDuplicates()
review_table.show(5)
review_table.count()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2SGG3U5KRZ1D3|    7735719|B00F1CPBQ0|     295169426| 2015-08-31|
|R2EFT92J89XYAO|   36369192|B0082I1IA8|     218693378| 2015-08-31|
|R2RWLQEQ6P6AO6|   16379976|B004CLBWY6|     900081003| 2015-08-31|
|R1H625FX48XDY0|   28543938|B0052QYLUM|     850036950| 2015-08-31|
|R35HCD443WGOAM|   50071436|B0012S9D4S|     202446354| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



1392128

In [44]:
# Create df for products table
products_df=cleaned_df.select(['product_id','product_title'])
products_df.show(5)
products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00GSP5D94|Summer Infant Swa...|
|B00BUBNZC8|Udder Covers - Br...|
|B00AWLZFTS|Gerber Graduates ...|
|B00KM60D3Q|Summer Infant Ult...|
|B00PQMRZG4|Summer Infant Kee...|
+----------+--------------------+
only showing top 5 rows



1392128

In [47]:
# Drop duplicates.
cleaned_products_df=products_df.dropDuplicates(['product_id'])
cleaned_products_df.show(5)
cleaned_products_df.count()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|6003003065|Home Decor Vinyl ...|
|B000056JES|LITTLE SUZY'S ZOO...|
|B000056OV2|My Best Friend Nu...|
|B00006JZ7J|Koala Baby - Cott...|
|B0000BUSS6|Recalled Item: Wi...|
+----------+--------------------+
only showing top 5 rows



139804

In [49]:
# Create df for vine table
vine_table_df = cleaned_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_table_df.show(5)
vine_table_df.count()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R8EWA1OFT84NX|          5|            0|          0|   N|
| RL5ESX231LZ0B|          5|            0|          0|   N|
| RRMS9ZWJ2KD08|          5|            0|          0|   N|
|R14I3ZG5E6S7YM|          5|            0|          0|   N|
|R13EPSFP5DODN5|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



1392128

In [50]:
# Drop Duplicates
vine_table_df=vine_table_df.dropDuplicates()
vine_table_df.show(5)
vine_table_df.count()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R11BXKH5WJER2O|          5|            0|          0|   N|
|R3PZ2Q9K8Y2C47|          5|            0|          0|   N|
|R2U3FC2KXQ3CI7|          5|            0|          0|   N|
|R26OPU2I25ZGS9|          5|            0|          0|   N|
| RF8MC62JYY86T|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



1392128

In [56]:
# Filter for Vine reviews.
Paid_df=vine_table_df.filter(vine_table_df['vine']== "Y")
Paid_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RSA6JQ346JZHZ|          5|           55|         64|   Y|
|R2SGI1S0QUSZFZ|          5|            0|          1|   Y|
|R2LO83W46LW8XV|          4|            0|          0|   Y|
| R6EPMKW17LWJE|          4|            0|          1|   Y|
|R2SJVUNVVT3PHD|          5|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [74]:
# Check summary statistics for Vine reviews.
Paid_df.describe(['star_rating', 'helpful_votes']).show()

+-------+-----------------+------------------+
|summary|      star_rating|     helpful_votes|
+-------+-----------------+------------------+
|  count|               19|                19|
|   mean|4.157894736842105| 3.473684210526316|
| stddev|0.898341551894183|12.509411661503474|
|    min|                2|                 0|
|    max|                5|                55|
+-------+-----------------+------------------+



In [67]:
# Check star count across star range for V.
v_star_count=Paid_df.groupBy('star_rating').count()
v_star_count.show()
v_star_count=v_star_count.withColumn('%',col('count')/19*100)
v_star_count.show()

+-----------+-----+
|star_rating|count|
+-----------+-----+
|          3|    3|
|          5|    8|
|          4|    7|
|          2|    1|
+-----------+-----+

+-----------+-----+------------------+
|star_rating|count|                 %|
+-----------+-----+------------------+
|          3|    3|15.789473684210526|
|          5|    8| 42.10526315789473|
|          4|    7| 36.84210526315789|
|          2|    1| 5.263157894736842|
+-----------+-----+------------------+



In [72]:
# Filter for !V reviews.
unPaid_df=vine_table_df.filter(vine_table_df['vine']== "N")
unPaid_df.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R11BXKH5WJER2O|          5|            0|          0|   N|
|R3PZ2Q9K8Y2C47|          5|            0|          0|   N|
|R2U3FC2KXQ3CI7|          5|            0|          0|   N|
|R26OPU2I25ZGS9|          5|            0|          0|   N|
| RF8MC62JYY86T|          4|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [73]:
# Check summary statistics for !V reviews.
unPaid_df.describe(['star_rating', 'helpful_votes']).show()

+-------+------------------+------------------+
|summary|       star_rating|     helpful_votes|
+-------+------------------+------------------+
|  count|           1392109|           1392109|
|   mean| 4.236202050270489|1.1019237717736183|
| stddev|1.2143407992166446|11.300942715637479|
|    min|                 1|                 0|
|    max|                 5|              5245|
+-------+------------------+------------------+



In [78]:
# Check star count across star range for !V.
nv_star_count=unPaid_df.groupBy('star_rating').count()
nv_star_count.show(5)
nv_star_count=nv_star_count.withColumn('%',col('count')/1392109*100)
nv_star_count.show()

+-----------+------+
|star_rating| count|
+-----------+------+
|          1| 93351|
|          3|119981|
|          5|876023|
|          4|229169|
|          2| 73585|
+-----------+------+

+-----------+------+------------------+
|star_rating| count|                 %|
+-----------+------+------------------+
|          1| 93351|6.7057249109085575|
|          3|119981| 8.618649832735798|
|          5|876023| 62.92775924873699|
|          4|229169| 16.46200117950534|
|          2| 73585| 5.285864828113316|
+-----------+------+------------------+



In [79]:
# Comparing V vs !V Star Count
nv_star_count.orderBy('star_rating').show()

v_star_count.orderBy('star_rating').show()

+-----------+------+------------------+
|star_rating| count|                 %|
+-----------+------+------------------+
|          1| 93351|6.7057249109085575|
|          2| 73585| 5.285864828113316|
|          3|119981| 8.618649832735798|
|          4|229169| 16.46200117950534|
|          5|876023| 62.92775924873699|
+-----------+------+------------------+

+-----------+-----+------------------+
|star_rating|count|                 %|
+-----------+-----+------------------+
|          2|    1| 5.263157894736842|
|          3|    3|15.789473684210526|
|          4|    7| 36.84210526315789|
|          5|    8| 42.10526315789473|
+-----------+-----+------------------+

